# LocalCart scenario part 2: Creating streams flows


## Introduction 

A web or mobile app will trigger events as a user navigates a web site. These clickstream events indicate when a customer logs in, adds something to a basket, completes an order, and logs out. The events are placed into configured Message Hub (Apache Kafka) that provides a scalable way to buffer the data before it is saved, analysed, and rendered. Using the instructions in [Notebook #1 - Creating a Kafka Producer of ClickStream events](https://github.com/wdp-beta/get-started/blob/beta_2/notebooks/localcart-scenario-part-1.ipynb) we generate clickstream events for LocalCart and send them to Message Hub to show how data can be collected offline and streamed to the cloud later. A Java app continuously feeds a simulated stream of events to Message Hub. 

This notebook is divided into two parts, describing how to use streams flows to perform streaming data analysis and save data for static analysis.


<img src="https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/nb2_flow.png"></img>


### Streaming data analysis

[Example 1: Capturing clickstream events for real-time analysis](#intro_a). You can use streaming pipelines to performs event-based aggregation operations (calculate the number of currently open baskets and value of those baskets, ...) on the fly and store the results into a Redis database. The aggregated data can easily be visualized in real-time using web applications that monitor this database, as described in [Notebook#4:Visualize streaming data in a real-time dashboard](https://github.com/wdp-beta/get-started/blob/beta_2/notebooks/localcart-scenario-part-4.ipynb). 


### Static data analysis

[Example 2: Capturing clickstream events for static analysis](#intro_b). You can also use streaming pipelines to store clickstream events (as-is or in modified form) in flat files, which can be processed offline - either by batch processes or interactively, as outlined in [Notebook#3b: Analyze static clickstreams](https://github.com/wdp-beta/get-started/blob/beta_2/notebooks/localcart-scenario-part-3b.ipynb).


This notebook runs on Python 2 with Spark 2.0.

<a id="intro_a"></a>

***
# Example 1: Capturing clickstream events for real-time analysis
***


<img src="https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/streaming_analysis.png"></img>


In this first example you will create a streams flow that ingests `login`, `add_to_basket` and `checkout` clickstream events, aggregates them according to our business needs and stores the aggregated data in a Redis database, which will be monitored by a real-time dashboard:

<img src='https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/MARMARMAR_result.png'></img>

## Example 1 table of contents

* [E1.1 Redis setup](#redis)<br>
* [E1.2 Create a streams flow](#create_p1) <br>
* [E1.3 Process login clickstream events](#login) <br>
* [E1.4 Process add_to_cart clickstream events](#addtocart) <br>
* [E1.5 Process checkout clickstream events](#checkout) <br>
* [E1.6 Run the flow](#run_1)<br>
* [E1.7 Summary and next steps](#summary_1)<br>


<a id="redis"></a>
***

## E1.1 Redis setup

Redis is an in-memory database. It stores its data in RAM, making it a very fast way of storing and retrieving data. It provides a set of primitive data structures, but we only concern ourselves with [hashes](https://redis.io/commands#hash) for this exercise.

A Redis hash is a data structure that allows several keys to be stored together. We are going to configure a Redis hash called `funnel` that contains the following output:

- login_count - the number of people who logged into LocalCart
- basket_count - the number of items added into a shopping cart
- checkout_count - the number of purchases made
- basket_total - the total price of items added into a shopping cart
- checkout_total - the total price of items purchased

These are the outputs of the aggregation functions in our streaming pipeline. 


### E1.1.1 Collect your Redis connection information

1. Open your <a target="_blank" href="https://apsportal.ibm.com/settings/services?context=analytics">IBM Cloud Data Services list</a>. A list of your provisioned services is displayed.
1. Locate the pre-provisioned **Compose for Redis** service and click on the service instance name.
1. Open the _Service Credentials_ tab and view the credentials.
```
{
  "db_type": "redis",
  "maps": [],
  "name": "b...b",
  "uri_cli": "redis-cli -h **HOSTNAME** -p **PORT** -a **PASSWORD**",
  "deployment_id": "5...2",
  "uri": "redis://admin:**PASSWORD**@**HOSTNAME**:**PORT**"
}
```

Note your `**HOSTNAME**`, `**PORT**` and `**PASSWORD**` information.


### E1.1.2 Verify your redis connectivity
You can verify your redis connectivity information in this notebook by installing the Python Redis library with the following command:

In [ ]:
!pip install redis

We import the library and connect to Redis with the following command. Replace the credential placeholders with your credentials.

In [ ]:
import redis
# TODO replace **HOSTNAME**, **PORT** and **PASSWORD** with your credentials
r = redis.StrictRedis(host='**HOSTNAME**', port=**PORT**, db=0, password='**PASSWORD**')

We can then create a hash called `funnel` to store our real-time data to the database by using the `hset` function:

In [ ]:
r.hset('funnel', 'basket_count', 554);
r.hset('funnel', 'basket_total', 951);
r.hset('funnel', 'checkout_count', 21);
r.hset('funnel', 'checkout_total', 5400);
r.hset('funnel', 'login_count', 100);

We can also use this connection to retrieve all the values from our `funnel` hash using `hgetall`:

In [ ]:
r.hgetall('funnel')

**Note:** 
The Redis connection above seems to freeze in this notebook after a minute or so. In this case, you will need to restart the notebook kernel to restore it.
<BR>
We can now create streams flows that store aggregated data in Redis.

<a id="create_p1"></a>
***

## E1.2 Create a streams flow

In IBM Watson Data Platform, do these steps:

1. Select a project that you want to contain the streams flows. Note that this project must be attached to Cloud Object Storage and not Object Storage (Swift).
1. Click the **Assets** tab and scroll to the _Streams flows_ section. (If no section with the name is displayed the selected project is not attached to Cloud Object Storage.)
1. Click **+ New streams flow**.
1. In the _New Streams Flow_ window, 
  1. Enter name `aggregate_for_redis`
  1. Select an existing Streaming Analytics service or create a new one (choosing the _Lite_ plan, which is free.) 
  1. Select **Manually**. (You will use the wizard in Example 2.)
  1. Click **Create**.

An empty canvas is displayed, along with a list of _Source_, _Target_, _Processing and Analytics_ and _Alerts_ operators that you can choose from. Source operators load data and target operators store data.

<a id="login"></a>
***

## E1.3 Process login clickstream events

First we need to collect `login` data from Message Hub and calculate the number of logins during a rolling one hour time window. The incoming `login` event payload has the following structure:
```
  {
    "customer_id": "13872",
    "click_event_type": "login",
    "total_price_of_basket": "0.0",
    "total_number_of_items_in_basket": "0",
    "total_number_of_distinct_items_in_basket": "0",
    "event_time": "2017-07-11 20:10:52 UTC"
  }
```


### E1.3.1 Configure the source

1. Drag a **MessageHub** source operator into the pipeline canvas.
1. Configure the MessageHub operator:
	1. Add a connection to your Message Hub instance.
	1. Select the `login` topic.
	1. Click **Edit Schema** to specify the payload properties this operator will make available to operators that are connected to its output port. Since we only want to count the number of login events we only make the `customer_id` available.
    1. Choose
            - Attribute Name: `customer_id`
            - Type: `Number` 
            - Path: `/customer_id` 
    1. Click **Save** and **Close**.         


Our streams flow now has its first operator and looks like this: 

<img src='https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/M.png'></img>


### E1.3.2 Set up aggregation functions

Streaming data can be aggregated by applying functions such as sum, count, minimum, or maximum. The results of the aggregation can be done on the aggregation before it is written to the Redis database. Our aim is to calculate the number of people who logged into LocalCart for a sliding one-hour window.

In the streams flow canvas, do these steps:

1. Drag an **Aggregation** operator from the _Processing and Analytics_ area, and then drop it on the canvas next to the Message Hub operator.
2. Drag your mouse pointer from the output port of the Message Hub operator to the input port of the Aggregation operator to connect them.
3. Click the **Aggregation** operator to open its _Properties_ pane. Set the following _Aggregation Window_ parameters:
    - Type - `sliding`
    - Time Units - `hour`
    - Number of Time Units - `1`
    - Partition By - leave unchanged
    - Group By - leave unchanged
4. In the **Functions** area of the _Aggregation Properties_ pane, define one aggregation:
    - Aggregation 1: count the logins
        - Output Field Name - `login_count`
        - Function Type - `Count`
        
    Note: To identify how many different customers have logged in during the rolling 1 hour time window, we would use the `CountDistinct` function and apply it to `customer_id`.

Our streams flow now has two connected operators: a source operator and an aggregation operator. Hover over the arrow to review the data flow between them.

<img src='https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/op_op_io.png'></img>



### E1.3.3 Configure the target

Next, add a Redis target operator. In the streams flow canvas, do these steps:

1. Drag a **Redis** operator from the _Target_ area, and then drop it on the canvas next to the Aggregation operator.
1. Drag your mouse pointer from the output port of the Aggregation operator to the input port of the Redis operator to connect them.
1. Click the **Redis** operator to open its Properties pane. 
    - Add a connection to your Redis instance.
      - Type in the `**HOST**`, `**PORT**` and `**PASSWORD**` credentials of your Compose for Redis service.
    - In the **Key Template** field, type in `funnel`. 
1. Save the streams flow. The setup for `login` event processing is complete.

  <img src='https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/MAR.png'></img>    



***

<a id="addtocart"></a>
## E1.4 Process add_to_cart clickstream events

Next we need to collect `add_to_cart` event data from Message Hub and calculate the number of shopping baskets and their combined value during a rolling one hour time window. The incoming `add_to_cart` event payload has the following structure:

```
{
    "customer_id": "13859",
    "click_event_type": "add_to_cart",
    "product_name": "Oatmeal",
    "product_category": "Food",
    "product_price": "2.49",
    "total_price_of_basket": "153.41",
    "total_number_of_items_in_basket": "19",
    "total_number_of_distinct_items_in_basket": "6",
    "event_time": "2017-06-23 12:56:18 UTC"
}
```

### E1.4.1 Configure the source, aggregation function and target for add_to_cart events

1. Drag another **Message Hub** source operator into the canvas.
1. Configure the Message Hub operator by doing these steps in the Properties pane:
	1. Select the Message Hub connection you've created earlier.
	1. Select the `add_to_cart` topic.
	1. Click **Edit Schema** to make the customer id and cart value available to connected operators. 
    1. The message schema can be automatically detected if a producer has already generated messages for the selected topic. Click **Detect Schema** and **Show preview**.
      > If no messsages are displayed and the schema is not populated verify that your producer is running.
    1. Click **Hide Preview**.
    1. Remove all attributes except `customer_id` and `total_price_of_basket`:
      - Attribute Name: `customer_id`
            - Type: `Number` 
            - JSON Path: `/customer_id` 
      - Attribute Name: `total_price_of_basket` 
            - Type: `Number` 
            - JSON Path: `/total_price_of_basket` 
    1. Click **Save** and **Close**.
1. Drag an **Aggregation** operator from the **Processing and Analytics** area, and then drop it on the canvas next to the Message Hub operator.
1. Drag your mouse pointer from the output port of the Message Hub operator to the input port of the Aggregation operator to connect them.
1. Click the **Aggregation** operator to open its _Properties_ pane. Set the following _Aggregation Window_ parameters:
    - Type - `sliding`
    - Time Units - `hour`
    - Number of Time Units - `1`
    - Partition By - leave unchanged
    - Group By - leave unchanged
1. In the **Functions** area of the _Aggregation Properties_ pane, define two aggregations:
    - Aggregation 1: count the baskets
        - Output Field Name - `basket_count`
        - Function Type - `Count`
    - Aggregation 2: Sum up basket values
        - Output Field Name - `basket_total`
        - Function Type - `Sum`
        - Apply Function to - `total_price_of_basket`
        
1. Copy the existing **Redis** operator that's already on the canvas and paste it next to the _Aggregation_ Operator. 
1. Drag your mouse pointer from the output port of the Aggregation operator to the input port of the Redis operator to connect them.

 Your pipeline is now configured to stream and aggregate `login` and `add_to_cart` events:
    
 <img src='https://raw.githubusercontent.com/wdp-beta/get-started/master/notebooks/images/MARMAR.png'></img>    

1. Save your streams flow. No errors should be reported.




<a id="checkout"></a>
***

## E1.5 Process checkout clickstream events

First we need to create a stream that collects `checkout` event data from a Message Hub operator and calculates the number of checkouts and their combined value during a rolling one hour time window. The incoming `checkout` event payload has the following structure:

```
{
    "customer_id": "11828",
    "click_event_type": "checkout",
    "total_price_of_basket": "72.80000000000001",
    "total_number_of_items_in_basket": "20",
    "total_number_of_distinct_items_in_basket": "5",
    "session_duration": "440",
    "event_time": "2017-06-23 13:09:12 UTC"
}
```

### E1.5.1 Set up pipeline source, aggregation function and target for checkout events

1. Drag another **Message Hub** source operator into the canvas.
1. Configure the MessageHub operator by doing these steps in the Properties pane:
	1. Select the ClickStream Message Hub connection.
	1. Select the `checkout` topic.
	1. Click **Edit Schema** to specify the message attributes that will be consumed. Define the following attributes (by entering them manually or customizing the auto-detected schema):
      - Attribute Name: `customer_id` 
            - Type: `Number` 
            - Path: `/customer_id` 
      - Attribute Name: `total_price_of_basket` 
            - Type: `Number` 
            - Path: `/total_price_of_basket` 
1. Drag an **Aggregation** operator from the _Processing and Analytics_ area, and then drop it on the canvas next to the Message Hub operator.
1. Drag your mouse pointer from the output port of the Message Hub operator to the input port of the Aggregation operator to connect them.
1. Click the **Aggregation** operator to open its _Properties_ pane. Set the following _Aggregation Window_ parameters:
    - Type - `sliding`
    - Time Units - `hour`
    - Number of Time Units - `1`
    - Partition By - leave unchanged
    - Group By - leave unchanged
1. In the **Functions** area of the _Aggregation Properties_ pane, define two aggregations:
    - Aggregation 1: count checkouts
        - Output Field Name - `checkout_count`
        - Function Type - `Count`
    - Aggregation 2: Sum basket values
        - Output Field Name - `checkout_total`
        - Function Type - `Sum`
        - Apply Function to - `total_price_of_basket`
        
1. Copy the existing **Redis** operator that's already on the canvas and paste it next to the _Aggregation_ Operator. 
1. Drag your mouse pointer from the output port of the Aggregation operator to the input port of the Redis operator to connect them. The completed stream now looks as follows: <br>
   <img src='https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/MARMARMAR.png'></img>    

1. Save the stream flow. No errors should be reported.

<a id="run_1"></a>
## E1.6 Run the stream flow

1. Click **Run**. 
1. If the flow does not start verify your stream flow. If no events are flowing from Message Hub operators make sure that your producer (simulating user activity), which you've launched in notebook 1, is running. 
1. Click on any operator to display throughput information.

<img src= "https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/nb2_redis_streaming_status.png"></img>

Congratulations! You just created a flow that ingests clickstream data from Message Hub, aggregates data and stores it in Redis storage.


<a id="summary_1"></a>
## E1.7 Summary and next steps
In this section, you consumed and aggregated clickstream events that were generated in [Notebook #1: Creating a Kafka Producer of ClickStream events](https://github.com/wdp-beta/get-started/blob/beta_2/notebooks/localcart-scenario-part-1.ipynb).

You can now skip to [Notebook#4:Visualize streaming data in a real-time dashboard](https://github.com/wdp-beta/get-started/blob/beta_2/notebooks/localcart-scenario-part-4.ipynb) to learn about how to visualize the aggregated data in real-time using a simple web application or continue to the next section to configure a pipeline for static analysis.

<img src="https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/nb2_dashboard.png"></img>



<a id="intro_b"></a>

***
# Example 2: Capturing clickstream events for static analysis
***


In this second example you will create multiple pipelines that ingest all clickstream events and store them as-is in CSV files on Cloud Object Storage in preparation for static analysis, as illustrated in [Notebook#3b: Analyze static clickstreams](https://github.com/wdp-beta/get-started/blob/beta_2/notebooks/localcart-scenario-part-3b.ipynb).

<img src="https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/static_analysis.png"></img>


## Example 2 table of contents

* [E2.1 Capture login clickstream events](#login_2) <br>
* [E2.2 Capture browsing clickstream events](#browsing_2) <br>
* [E2.3 Capture add_to_cart clickstream events](#atc_2) <br>
* [E2.4 Capture checkout clickstream events](#checkout_2) <br>
* [E2.5 Capture logout_with_purchase clickstream events](#lwp_2) <br>
* [E2.6 Capture logout_without_purchase clickstream events](#lwop_2) <br>
* [E2.7 Summary and next steps](#summary_2)<br>


<a id="login_2"></a>
***

## E2.1 Create a flow for login clickstream events

In IBM Watson Data Platform, do these steps:

1. Select a project that you want to contain the flow.
1. Click the **Assets** tab and scroll to the _Streams flows_ section.
1. Click **+ New streams flow**.
1. In the _New Streams Flow_ window, 
  1. Enter streams flow name `store_events_on_cos`.
  1. Select **Wizard**. 
  1. Click **Create**.
1. In the _Select Source_ tab click **Message Hub**.
1. Under the Connection drop-down menu, select your Message Hub connection.
1. Under the Topic drop-down menu, select **login** and click **Continue**.
1. Wait for the Data Preview window to display the streaming data for the selected event. (If no data is displayed make sure your producer is running.)
 > You can customize the pre-defined schema (e.g. remove attributes or change data types) by clicking _Edit schema_. In this scenario we want to store the complete event payload, therefore no changes are required. 
1. Click **Continue**.
1. In the Select Target window, click **Cloud Object Storage**.
1. Under the Cloud Object Storage connection drop-down menu, select the instance that is used by the project.
   <br>
   > Take note of the Cloud Object Storage connection name. You will need this information in [Notebook 3b: Static clickstream analysis](https://github.com/wdp-beta/get-started/blob/beta_2/notebooks/localcart-scenario-part-3b.ipynb) when you load and analyze the clickstream events.
1. Under _File path_, select an existing bucket and append file name **login_%TIME.csv**. Your file path should look similar to this: `/existing-bucket-name/login_%TIME.csv`. (**Note:** "TIME" is a reserved word that will be replaced with an actual timestamp when the file is written).
   > Note: if you choose a file name other than `login_%TIME.csv` you must also modify notebook 3B and change the default file name in the data load cell.
1. Under _Format_, select **csv**.
1. Under _Delimiter_, select **Comma (,)**.
1. Under _File Creation Policy_ choose **Time** and **60** seconds.
   > Note: Always choose a creation policy that apropriately reflects your data throughput and persistence requirements.
1. Click **Continue** and **Save**. 
1. Click **Run** to start the streams flow. If no errors are reported all login events are written to timestamped CSV files in Cloud Object Storage.
      <img src="https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/nb2_csv_login_monitoring.png"></img>
1. Click **Stop** to stop the streams flow.

> You can backup a flow by downloading/exporting it.

Open the the streams flow editor again to add the processing logic for the other clickstream event types.

<a id="browsing_2"></a>
***

## E2.2 Capture browsing clickstream events

1. Copy and paste the existing **Message Hub** operator.
1. Configure the new operator properties as follows:
 1. Rename the operator to LC_BRWS to make it easily distinguishable from other Message Hub operators.
  > You can choose any name.
 1. Keep the existing connection.
 1. Select the **browsing** topic.
 1. Edit the schema by detecting it. (You will be prompted to override the current definition.)
1. Copy and paste the existing **Cloud Object Storage** operator.
1. Connect the output port of the Message Hub operator with the input port of the Cloud Object Storage operator.
1. Configure the new operator properties as follows:
 1. If desired, rename the operator. 
 1. Keep the existing Cloud Object Storage connection.
 1. Under _File path_, select an existing bucket and append file name **browsing_%TIME.csv**. Your file path should look similar to this: `/existing-bucket-name/browsing_%TIME.csv`. 
 1. Under _Format_, select **csv**.
 1. Under _Delimiter_, select **Comma (,)**.
 1. Under _File Creation Policy_ choose **Time** and **60** seconds.
1. Save the flow. 


<a id="atc_2"></a>
***

## E2.3 Capture add_to_cart clickstream events

1. Copy and paste one of the existing **Message Hub** operators.
1. Configure the new operator properties as follows:
 1. Rename the operator to LC_ADD to make it easily distinguishable from other Message Hub operators.
 1. Keep the existing connection.
 1. Select the **add_to_cart** topic.
 1. Edit the schema by detecting it. (You will be prompted to override the current definition.)
1. Copy and paste one of the existing **Cloud Object Storage** operators.
1. Connect the output port of the new Message Hub operator with the input port of the new Cloud Object Storage operator.
1. Configure the new Cloud Object Storage operator properties as follows:
 1. If desired, rename the operator. 
 1. Keep the existing Cloud Object Storage connection.
 1. Under _File path_, select an existing bucket and append file name **addtocart_%TIME.csv**. Your file path should look similar to this: `/existing-bucket-name/addtocart_%TIME.csv`. 
 1. Under _Format_, select **csv**.
 1. Under _Delimiter_, select **Comma (,)**.
 1. Under _File Creation Policy_ choose **Time** and **60** seconds.
1. Save the flow. 


<a id="checkout_2"></a>
***

## E2.4 Capture checkout clickstream events

1. Copy and paste one of the existing **Message Hub** operators.
1. Configure the operator properties as follows:
 1. Rename the operator to LC_CHKO to make it easily distinguishable from other Message Hub operators.
 1. Keep the existing connection.
 1. Select the **checkout** topic.
 1. Edit the schema by detecting it. (You will be prompted to override the current definition.)
1. Copy and paste one of the existing **Cloud Object Storage** operators.
1. Connect the output port of the new Message Hub operator with the input port of the new Cloud Object Storage operator.
1. Configure the new Cloud Object Storage operator properties as follows:
 1. If desired, rename the operator. 
 1. Keep the existing Cloud Object Storage connection.
 1. Under _File path_, select an existing bucket and append file name **checkout_%TIME.csv**. Your file path should look similar to this: `/existing-bucket-name/checkout_%TIME.csv`. 
 1. Under _Format_, select **csv**.
 1. Under _Delimiter_, select **Comma (,)**.
 1. Under _File Creation Policy_ choose **Time** and **60** seconds.
1. Save the flow. 


<a id="lwp_2"></a>
***

## E2.5 Capture logout_with_purchase clickstream events

1. Copy and paste one of the existing **Message Hub** operators.
1. Configure the operator properties as follows:
 1. Rename the operator to LC_PRCH to make it easily distinguishable from other Message Hub operators.
 1. Keep the existing connection.
 1. Select the **logout_with_purchase** topic.
 1. Edit the schema by detecting it. (You will be prompted to override the current definition.)
1. Copy and paste one of the existing **Cloud Object Storage** operators.
1. Connect the output port of the new Message Hub operator with the input port of the new Cloud Object Storage operator.
1. Configure the new Cloud Object Storage operator properties as follows:
 1. If desired, rename the operator. 
 1. Keep the existing Cloud Object Storage connection.
 1. Under _File path_, select an existing bucket and append file name **logoutwithpurchase_%TIME.csv**. Your file path should look similar to this: `/existing-bucket-name/logoutwithpurchase_%TIME.csv`. 
 1. Under _Format_, select **csv**.
 1. Under _Delimiter_, select **Comma (,)**.
 1. Under _File Creation Policy_ choose **Time** and **60** seconds.
1. Save the flow. 

<a id="lwop_2"></a>
***

## E2.6 Capture logout_without_purchase clickstream events

1. Copy and paste one of the existing **Message Hub** operators.
1. Configure the operator properties as follows:
 1. Rename the operator to LC_NOPRCH to make it easily distinguishable from other Message Hub operators.
 1. Keep the existing connection.
 1. Select the **logout_without_purchase** topic.
 1. Edit the schema by detecting it. (You will be prompted to override the current definition.)
1. Copy and paste one of the existing **Cloud Object Storage** operators.
1. Connect the output port of the new Message Hub operator with the input port of the new Cloud Object Storage operator.
1. Configure the new Cloud Object Storage operator properties as follows:
 1. If desired, rename the operator. 
 1. Keep the existing Cloud Object Storage connection.
 1. Under _File path_, select an existing bucket and append file name **logoutwithoutpurchase_%TIME.csv**. Your file path should look similar to this: `/existing-bucket-name/logoutwithoutpurchase_%TIME.csv`. 
 1. Under _Format_, select **csv**.
 1. Under _Delimiter_, select **Comma (,)**.
 1. Under _File Creation Policy_ choose **Time** and **60** seconds.
1. Save the flow. 

  <img src="https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/nb2_store_to_cos_stream.png"></img>


1. Click **Run**. Six flows should be running, capturing `add_to_cart`, `browsing`, `checkout`, `login`, `logout_with_purchase`, and `logout_without_purchase` events and storing them in CSV files on Cloud Object Storage:
 <img src="https://raw.githubusercontent.com/wdp-beta/get-started/beta_2/notebooks/images/nb2_csv_flows_monitoring.png"></img>


<a id="summary_2"></a>
***

## E2.7 Summary and next steps

You can now do one of the following:

#### Accessing CSV files on Cloud Object Storage
1. Open your <a target="_blank" href="https://apsportal.ibm.com/settings/services?context=analytics">IBM Cloud Data Services list</a>. A list of your provisioned services is displayed.
1. Locate the pre-provisioned Cloud Object Storage service and click on the service instance name.

#### Accessing CSV files on Cloud Object Storage manually
1. Open the **Buckets and objects** tab, and then select the bucket that you specified when you configured the Cloud Object Storage target operators. 
1. Click on the download link next to a CSV file.

#### Accessing CSV files on Cloud Object Storage programatically
1. Open the **Service credentials** tab. Select a Key Name, and then click **View credentials**. 
1. Copy the credentials and provide this information whenever you want to load data files programatically, such as in [Notebook 3b: Static clickstream analysis](https://github.com/wdp-beta/get-started/blob/beta_2/notebooks/localcart-scenario-part-3b.ipynb).



***

### Authors

Glynn Bird is a Developer Advocate for Watson Data Platform at IBM. 

Raj Singh is a Developer Advocate for Watson Data Platform at IBM.

***
Copyright © IBM Corp. 2017, 2018. This notebook and its source code are released under the terms of the MIT License.